### Get available subreddits list

In [2]:
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
import time
import random
from googletrans import Translator

SUBREDDIT_LIST_PAGE = 'https://np.reddit.com/r/Turkiye/wiki/turkishsubsindexv2'

In [3]:
headers = {
  'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; rv:91.0) Gecko/20100101 Firefox/91.0'
}

#page = requests.get(SUBREDDIT_LIST_PAGE, headers=headers)

In [42]:
def wRequest(path):
    time.sleep(1)
    return requests.get(path, headers=headers)

In [6]:
def get_subreddits(content):
    subreddits = []
    
    soup = BeautifulSoup(content, 'html.parser')
    for table in soup.find_all('table'):
        if 'Sub' in table.thead.th.contents[0]:
            for data in table.find_all('td'):
                if re.match(r'^/r/', data.text):
                     subreddits.append(f"https://np.reddit.com{data.text}")
    return subreddits

In [7]:
def get_post_addresses(subreddit):
    post_addresses = []
    
    search_subreddit = str(subreddit)
    last_post_fullname = None
    while subreddit:
        print(f'{100*" "}\r# of addresses found -> {len(post_addresses)}', end='\r')
        post_list_page = wRequest(search_subreddit)
        post_page_soup = BeautifulSoup(post_list_page.content)

        posts = post_page_soup.find('div', id='siteTable')

        for post in posts.find_all('div'):
            if post.has_attr('data-comments-count') and int(post['data-comments-count']) > 0:
                post_addresses.append(f"https://np.reddit.com{post['data-permalink']}")

            if post.has_attr('data-fullname'):
                post_fullname = post['data-fullname']
        
        if last_post_fullname != post_fullname:
            last_post_fullname = post_fullname
        else:
            break
            
        search_subreddit = f'{subreddit}/?count=100&after={last_post_fullname}'
    return post_addresses


In [16]:
def write_post_addresses(subreddit, filename):
    with open(filename, 'a') as f:
        search_subreddit = str(subreddit)
        last_post_fullname = None
        while subreddit:
            post_list_page = wRequest(search_subreddit)
            post_page_soup = BeautifulSoup(post_list_page.content)

            posts = post_page_soup.find('div', id='siteTable')

            for post in posts.find_all('div'):
                if post.has_attr('data-comments-count') and int(post['data-comments-count']) > 0:
                    f.write(f"https://np.reddit.com{post['data-permalink']}\n")

                if post.has_attr('data-fullname'):
                    post_fullname = post['data-fullname']

            if last_post_fullname != post_fullname:
                last_post_fullname = post_fullname
            else:
                break

            search_subreddit = f'{subreddit}/?count=100&after={last_post_fullname}'


In [7]:
def get_post_response(content):
    soup = BeautifulSoup(content, 'html.parser')
    entries = []
    for e in soup.find_all(class_="entry unvoted"):  
        score_obj = e.find(class_='score likes')
        score = int(score_obj.text.split()[0]) if score_obj else -1
        found_obj = e.find(class_='md')
        comment = found_obj.text if found_obj else None
        
        if comment is not None:
            entries.append((comment, score))
    
    if len(entries) > 1:
        post = entries[0][0].strip()
        golden_response = max(entries[1:], key=lambda x:x[1])[0].strip()

        return post, golden_response

    else:
        return None, None

In [20]:
# append to my_text.txt file
subreddits = get_subreddits(page.content)
for i, subreddit in tqdm(enumerate(subreddits, start=1)):
    try:
        write_post_addresses(subreddit, '___.txt')
        
    except KeyboardInterrupt:
        break
        
    except:
        pass

780it [6:36:15, 30.48s/it] 


In [4]:
len(posts), len(failed_posts), len(worked_posts)

(67862, 2765, 1296)

In [27]:
pdetector = Translator()

with open('failed.txt') as f:
    failed_posts = [line[:-1] for line in f.readlines()]

with open('worked.txt') as f:
    worked_posts = [line[:-1] for line in f.readlines()]
    
with open('my_text.txt') as f:
    posts = []
    for line in f.readlines():
        line = line[:-1]
        
        if line not in failed_posts and line not in worked_posts:
            posts.append(line)
            
post_resp_dict = {'link':[], 'post':[], 'response':[]}
    
for i, post_addr in enumerate(tqdm(posts)):
    start_time = time.time()
    post_page = requests.get(post_addr, headers=headers)
    
    post, response = get_post_response(post_page.content)
    
    if post in failed_posts:
        continue
        
    if post and response and detector.detect(post).lang == 'tr' and detector.detect(response).lang == 'tr':
            post_resp_dict['link'].append(post_addr)
            post_resp_dict['post'].append(post)
            post_resp_dict['response'].append(response)

            with open('worked.txt', 'a') as f:
                f.write(f'{post_addr}\n')
    else:
        with open('failed.txt', 'a') as f:
            f.write(f'{post_addr}\n')
            
    while time.time() - start_time < 4:
        pass
        

  3%|███                                                                                                            | 1903/67832 [2:09:43<74:54:33,  4.09s/it]


ConnectTimeout: timed out